In [1]:
from google.colab import drive
import pandas as pd
import math
from collections import Counter
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
file_path_train = '/content/drive/MyDrive/cs5350/hw5/train.csv'
file_path_test = '/content/drive/MyDrive/cs5350/hw5/test.csv'
attributes_train = []
labels_train = []
attributes_test = []
labels_test = []
with open(file_path_train, 'r') as f:
    for line in f:
        terms = line.strip().split(',')
        attr = [float(x) for x in terms[:4]]
        label = int(terms[4])
        attributes_train.append(attr)
        labels_train.append(label)

with open(file_path_test, 'r') as f:
    for line in f:
        terms = line.strip().split(',')
        attr = [float(x) for x in terms[:4]]
        label = int(terms[4])
        attributes_test.append(attr)
        labels_test.append(label)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
X_train = np.array(attributes_train, dtype=float)
y_train = np.array(labels_train, dtype=int)
X_test = np.array(attributes_test, dtype=float)
y_test = np.array(labels_test, dtype=int)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

a)

In [15]:
import numpy as np
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [16]:
def sigmoid_prime(z):
    s = sigmoid(z)
    return s * (1 - s)

In [17]:
class ThreeLayerNetwork:
    def __init__(self):
        #Layer 1
        self.w1_01 = -1.0
        self.w1_11 = -2.0
        self.w1_21 = -3.0
        self.w1_02 = 1.0
        self.w1_12 = 2.0
        self.w1_22 = 3.0

        #2
        self.w2_01 = -1.0
        self.w2_11 = -2.0
        self.w2_21 = -3.0
        self.w2_02 = 1.0
        self.w2_12 = 2.0
        self.w2_22 = 3.0

        #3
        self.w3_01 = -1.0
        self.w3_11 = 2.0
        self.w3_21 = -1.5

    def forward(self, x1, x2):
        x0 = 1.0
        #1st hidden layer
        a1_1 = self.w1_01*x0 + self.w1_11*x1 + self.w1_21*x2
        a1_2 = self.w1_02*x0 + self.w1_12*x1 + self.w1_22*x2
        z1_1 = sigmoid(a1_1)
        z1_2 = sigmoid(a1_2)
        z1_0 = 1.0

        #2nd
        a2_1 = self.w2_01*z1_0 + self.w2_11*z1_1 + self.w2_21*z1_2
        a2_2 = self.w2_02*z1_0 + self.w2_12*z1_1 + self.w2_22*z1_2
        z2_1 = sigmoid(a2_1)
        z2_2 = sigmoid(a2_2)
        z2_0 = 1.0

        #Output
        a3_1 = self.w3_01*z2_0 + self.w3_11*z2_1 + self.w3_21*z2_2
        y = sigmoid(a3_1)

        cache = {
            'x0': x0, 'x1': x1, 'x2': x2,
            'a1_1': a1_1, 'a1_2': a1_2,
            'z1_0': z1_0, 'z1_1': z1_1, 'z1_2': z1_2,
            'a2_1': a2_1, 'a2_2': a2_2,
            'z2_0': z2_0, 'z2_1': z2_1, 'z2_2': z2_2,
            'a3_1': a3_1, 'y': y
        }
        return y, cache

    def backward(self, cache, y_true):
        y = cache['y']
        dL_dy = (y - y_true)
        dL_da3_1 = dL_dy * sigmoid_prime(cache['a3_1'])

        z2_0, z2_1, z2_2 = cache['z2_0'], cache['z2_1'], cache['z2_2']
        dL_dw3_01 = dL_da3_1 * z2_0
        dL_dw3_11 = dL_da3_1 * z2_1
        dL_dw3_21 = dL_da3_1 * z2_2

        dL_dz2_1 = dL_da3_1 * self.w3_11
        dL_dz2_2 = dL_da3_1 * self.w3_21

        da2_1 = sigmoid_prime(cache['a2_1'])
        da2_2 = sigmoid_prime(cache['a2_2'])

        dL_da2_1 = dL_dz2_1 * da2_1
        dL_da2_2 = dL_dz2_2 * da2_2

        z1_0, z1_1, z1_2 = cache['z1_0'], cache['z1_1'], cache['z1_2']
        dL_dw2_01 = dL_da2_1 * z1_0
        dL_dw2_11 = dL_da2_1 * z1_1
        dL_dw2_21 = dL_da2_1 * z1_2

        dL_dw2_02 = dL_da2_2 * z1_0
        dL_dw2_12 = dL_da2_2 * z1_1
        dL_dw2_22 = dL_da2_2 * z1_2

        dL_dz1_1 = dL_da2_1*self.w2_11 + dL_da2_2*self.w2_12
        dL_dz1_2 = dL_da2_1*self.w2_21 + dL_da2_2*self.w2_22

        da1_1 = sigmoid_prime(cache['a1_1'])
        da1_2 = sigmoid_prime(cache['a1_2'])

        dL_da1_1 = dL_dz1_1 * da1_1
        dL_da1_2 = dL_dz1_2 * da1_2

        x0, x1, x2 = cache['x0'], cache['x1'], cache['x2']
        dL_dw1_01 = dL_da1_1 * x0
        dL_dw1_11 = dL_da1_1 * x1
        dL_dw1_21 = dL_da1_1 * x2

        dL_dw1_02 = dL_da1_2 * x0
        dL_dw1_12 = dL_da1_2 * x1
        dL_dw1_22 = dL_da1_2 * x2

        grads = {
            'w3_01': dL_dw3_01, 'w3_11': dL_dw3_11, 'w3_21': dL_dw3_21,
            'w2_01': dL_dw2_01, 'w2_11': dL_dw2_11, 'w2_21': dL_dw2_21,
            'w2_02': dL_dw2_02, 'w2_12': dL_dw2_12, 'w2_22': dL_dw2_22,
            'w1_01': dL_dw1_01, 'w1_11': dL_dw1_11, 'w1_21': dL_dw1_21,
            'w1_02': dL_dw1_02, 'w1_12': dL_dw1_12, 'w1_22': dL_dw1_22
        }
        return grads

net = ThreeLayerNetwork()
x1, x2 = 1.0, 1.0
y_true = 1.0

y_pred, cache = net.forward(x1, x2)
print("Output y:", y_pred)

grads = net.backward(cache, y_true)
print("Gradients:")
for k, v in grads.items():
    print(k, v)

Output y: 0.08040217301208424
Gradients:
w3_01 -0.06799291476718632
w3_11 -0.0012259078520512152
w3_21 -0.06676700691513511
w2_01 -0.0024076096256642077
w2_11 -5.9531113125541e-06
w2_21 -0.002401656514351654
w2_02 0.001805707219248146
w2_12 4.464833484415552e-06
w2_22 0.0018012423857637306
w1_01 2.0784370290840247e-05
w1_11 2.0784370290840247e-05
w1_21 2.0784370290840247e-05
w1_02 3.117655543625889e-05
w1_12 3.117655543625889e-05
w1_22 3.117655543625889e-05


In [18]:
net = ThreeLayerNetwork()
x1, x2 = 1.0, 1.0
y_true = 1.0

y_pred, cache = net.forward(x1, x2)
print("Output y:", y_pred)

grads = net.backward(cache, y_true)
print("Gradients:")
for k, v in grads.items():
    print(k, v)

Output y: 0.08040217301208424
Gradients:
w3_01 -0.06799291476718632
w3_11 -0.0012259078520512152
w3_21 -0.06676700691513511
w2_01 -0.0024076096256642077
w2_11 -5.9531113125541e-06
w2_21 -0.002401656514351654
w2_02 0.001805707219248146
w2_12 4.464833484415552e-06
w2_22 0.0018012423857637306
w1_01 2.0784370290840247e-05
w1_11 2.0784370290840247e-05
w1_21 2.0784370290840247e-05
w1_02 3.117655543625889e-05
w1_12 3.117655543625889e-05
w1_22 3.117655543625889e-05


b)

In [20]:
class ThreeLayerNetwork:
    def __init__(self, input_size, hidden_size, output_size=1):
        self.W1 = np.random.randn(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.randn(hidden_size, hidden_size)
        self.b2 = np.zeros(hidden_size)
        self.W3 = np.random.randn(hidden_size, output_size)
        self.b3 = np.zeros(output_size)

    def forward(self, X):
        z1 = X.dot(self.W1) + self.b1
        a1 = sigmoid(z1)

        z2 = a1.dot(self.W2) + self.b2
        a2 = sigmoid(z2)

        z3 = a2.dot(self.W3) + self.b3
        y = sigmoid(z3)

        cache = {
            'X': X, 'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2, 'z3': z3, 'y': y
        }
        return y, cache

    def loss(self, X, y_true):
        y, cache = self.forward(X)

        diff = (y_true.reshape(-1,1) - y)
        loss = 0.5 * np.mean(diff**2)

        N = X.shape[0]
        dL_dy = (y - y_true.reshape(-1,1))
        dL_dz3 = dL_dy * sigmoid_prime(cache['z3'])

        dW3 = cache['a2'].T.dot(dL_dz3) / N
        db3 = np.sum(dL_dz3, axis=0) / N

        dL_da2 = dL_dz3.dot(self.W3.T)
        dL_dz2 = dL_da2 * sigmoid_prime(cache['z2'])

        dW2 = cache['a1'].T.dot(dL_dz2) / N
        db2 = np.sum(dL_dz2, axis=0) / N

        dL_da1 = dL_dz2.dot(self.W2.T)
        dL_dz1 = dL_da1 * sigmoid_prime(cache['z1'])

        dW1 = cache['X'].T.dot(dL_dz1) / N
        db1 = np.sum(dL_dz1, axis=0) / N

        grads = {
            'W1': dW1, 'b1': db1,
            'W2': dW2, 'b2': db2,
            'W3': dW3, 'b3': db3
        }
        return loss, grads

    def update_params(self, grads, lr):
        self.W1 -= lr * grads['W1']
        self.b1 -= lr * grads['b1']
        self.W2 -= lr * grads['W2']
        self.b2 -= lr * grads['b2']
        self.W3 -= lr * grads['W3']
        self.b3 -= lr * grads['b3']

    def predict(self, X):
        y_pred, _ = self.forward(X)
        return np.where(y_pred >= 0.5, 1, -1).flatten()

In [21]:
def compute_error(y_pred, y_true):
    return np.mean(y_pred != y_true)

In [22]:
def train_network(X_train, y_train, X_test, y_test, hidden_size, gamma0, d, epochs=10, batch_size=32):
    N = X_train.shape[0]
    net = ThreeLayerNetwork(X_train.shape[1], hidden_size)

    updates = 0
    train_loss_history = []

    for epoch in range(epochs):
        perm = np.random.permutation(N)
        X_train = X_train[perm]
        y_train = y_train[perm]

        for i in range(0, N, batch_size):
            X_batch = X_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            loss, grads = net.loss(X_batch, y_batch)
            lr = gamma0 / (1 + (gamma0/d)*updates)
            net.update_params(grads, lr)

            updates += 1
            train_loss_history.append(loss)

        train_pred = net.predict(X_train)
        test_pred = net.predict(X_test)
        train_err = compute_error(train_pred, y_train)
        test_err = compute_error(test_pred, y_test)
        print(f"Epoch {epoch+1}, Updates {updates}, Loss {loss:.4f}, Train Err: {train_err:.4f}, Test Err: {test_err:.4f}")

    return net, train_loss_history

In [24]:
gamma0 = 0.1
d = 1000
hidden_sizes = [5, 10, 25, 50, 100]
for h in hidden_sizes:
    print(f"Training with hidden size {h}")
    net, loss_hist = train_network(X_train, y_train, X_test, y_test, h, gamma0, d, epochs=20, batch_size=32)

Training with hidden size 5
Epoch 1, Updates 28, Loss 0.4332, Train Err: 0.2890, Test Err: 0.2620
Epoch 2, Updates 56, Loss 0.6024, Train Err: 0.4461, Test Err: 0.4420
Epoch 3, Updates 84, Loss 0.5264, Train Err: 0.4461, Test Err: 0.4420
Epoch 4, Updates 112, Loss 0.5560, Train Err: 0.4461, Test Err: 0.4420
Epoch 5, Updates 140, Loss 0.5637, Train Err: 0.4461, Test Err: 0.4420
Epoch 6, Updates 168, Loss 0.4952, Train Err: 0.4461, Test Err: 0.4420
Epoch 7, Updates 196, Loss 0.4295, Train Err: 0.4461, Test Err: 0.4420
Epoch 8, Updates 224, Loss 0.4038, Train Err: 0.4461, Test Err: 0.4420
Epoch 9, Updates 252, Loss 0.4461, Train Err: 0.4461, Test Err: 0.4420
Epoch 10, Updates 280, Loss 0.4826, Train Err: 0.4461, Test Err: 0.4420
Epoch 11, Updates 308, Loss 0.5524, Train Err: 0.4461, Test Err: 0.4420
Epoch 12, Updates 336, Loss 0.4201, Train Err: 0.4461, Test Err: 0.4420
Epoch 13, Updates 364, Loss 0.4577, Train Err: 0.4461, Test Err: 0.4420
Epoch 14, Updates 392, Loss 0.3622, Train Err: 0

c) After changing the weights to 0, it seemed as if the errors platued and this is probably because it stops learning.

d)The neural network has lower test error than the SVM and better preformance but SVM is more stable.

e) some credit to Deep Learning w/Anna Fariha assignments

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

y_train_binary = torch.where(y_train == 1, torch.tensor(1.0), torch.tensor(0.0))
y_test_binary = torch.where(y_test == 1, torch.tensor(1.0), torch.tensor(0.0))


<ipython-input-35-d0dbac6c34bf>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
<ipython-input-35-d0dbac6c34bf>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-35-d0dbac6c34bf>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
<ipython-input-35-d0dbac6c34bf>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourc

In [28]:
def initialize_weights(m, activation='tanh'):
    if isinstance(m, nn.Linear):
        if activation == 'tanh':
            nn.init.xavier_normal_(m.weight)
        elif activation == 'relu':
            nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

In [29]:
def build_network(input_size, depth, width, activation='tanh'):
    layers = []
    layers.append(nn.Linear(input_size, width))
    if activation == 'tanh':
        layers.append(nn.Tanh())
    else:
        layers.append(nn.ReLU())

    for _ in range(depth - 2):
        layers.append(nn.Linear(width, width))
        if activation == 'tanh':
            layers.append(nn.Tanh())
        else:
            layers.append(nn.ReLU())

    layers.append(nn.Linear(width, 1))

    model = nn.Sequential(*layers)
    model.apply(lambda m: initialize_weights(m, activation))
    return model

In [33]:
def compute_error(model, X, y):
    model.eval()
    with torch.no_grad():
        logits = model(X)
        pred = torch.sigmoid(logits)
        pred_labels = (pred >= 0.5).float()
        y_binary = torch.where(y == 1, torch.tensor(1.0, device=y.device), torch.tensor(0.0, device=y.device))
        error = (pred_labels.flatten() != y_binary).float().mean().item()
    return error


In [31]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32):
    y_train_binary = torch.where(y_train == 1, torch.tensor(1.0, device=y_train.device), torch.tensor(0.0, device=y_train.device))
    y_test_binary = torch.where(y_test == 1, torch.tensor(1.0, device=y_test.device), torch.tensor(0.0, device=y_test.device))

    dataset = TensorDataset(X_train, y_train_binary)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            logits = model(X_batch)
            loss = criterion(logits, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()

    train_err = compute_error(model, X_train, y_train)
    test_err = compute_error(model, X_test, y_test)
    return train_err, test_err

In [37]:
depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
activations = ['tanh', 'relu']

for activation in activations:
    print(f"Activation: {activation}")
    for depth in depths:
        for width in widths:
            model = build_network(input_size=X_train.shape[1], depth=depth, width=width, activation=activation)
            train_err, test_err = train_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32)
            print(f"Depth: {depth}, Width: {width}, Train Err: {train_err:.4f}, Test Err: {test_err:.4f}")

Activation: tanh
Depth: 3, Width: 5, Train Err: 0.0161, Test Err: 0.0160
Depth: 3, Width: 10, Train Err: 0.0000, Test Err: 0.0000
Depth: 3, Width: 25, Train Err: 0.0000, Test Err: 0.0000
Depth: 3, Width: 50, Train Err: 0.0000, Test Err: 0.0000
Depth: 3, Width: 100, Train Err: 0.0000, Test Err: 0.0000
Depth: 5, Width: 5, Train Err: 0.0000, Test Err: 0.0040
Depth: 5, Width: 10, Train Err: 0.0000, Test Err: 0.0020
Depth: 5, Width: 25, Train Err: 0.0000, Test Err: 0.0000
Depth: 5, Width: 50, Train Err: 0.0000, Test Err: 0.0000
Depth: 5, Width: 100, Train Err: 0.0000, Test Err: 0.0000
Depth: 9, Width: 5, Train Err: 0.0000, Test Err: 0.0020
Depth: 9, Width: 10, Train Err: 0.0000, Test Err: 0.0000
Depth: 9, Width: 25, Train Err: 0.0000, Test Err: 0.0000
Depth: 9, Width: 50, Train Err: 0.0000, Test Err: 0.0000
Depth: 9, Width: 100, Train Err: 0.0000, Test Err: 0.0000
Activation: relu
Depth: 3, Width: 5, Train Err: 0.0654, Test Err: 0.0660
Depth: 3, Width: 10, Train Err: 0.0034, Test Err: 0.006

The super low or non existent values are super suspicous and I conclude I did this wrong.

3.a)

In [38]:
%%writefile script.sh

Writing script.sh


In [39]:
%%writefile run.sh

Writing run.sh
